<a href="https://colab.research.google.com/github/Rijan4449/XGB-OOA/blob/main/XGOOA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import time
import random

from collections import Counter
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, log_loss, classification_report, confusion_matrix
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
import xgboost as xgb
from xgboost.callback import EarlyStopping
from imblearn.over_sampling import SMOTE
import math
from copy import deepcopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -----------------------
# 0. Seed
# -----------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
# -----------------------
# 1. Load dataset
# -----------------------
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/super_dataset.csv")

# PH waters test set
with open("/content/drive/My Drive/Colab Notebooks/ph_waters.txt", "r", encoding="utf-8") as f:
    ph_waters = {line.strip() for line in f if line.strip()}

ph_test = df[df["waterbody_name"].isin(ph_waters)]
non_ph = df[~df["waterbody_name"].isin(ph_waters)]
test_size = int(0.2 * len(df))
if len(ph_test) < test_size:
    additional_needed = test_size - len(ph_test)
    extra_non_ph = non_ph.sample(n=additional_needed, random_state=42)
    test_set = pd.concat([ph_test, extra_non_ph])
    train_set = df.drop(test_set.index)
else:
    test_set = ph_test
    train_set = non_ph

In [ ]:
# -----------------------
# 2. Define columns
# -----------------------
drop_cols = ['fish_id', 'common_name', 'status']  # note: we still drop 'status' later for final train
high_cardinality = ['species', 'waterbody_name']
low_cardinality = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'feeding_type']
numeric_cols = [
    'temp_max', 'weight_max', 'length_max', 'temp_pref_min', 'temp_pref_max',
    'fecundity_mean', 'fecundity_min', 'fecundity_max',
    'trophic_lvl_estimate_min', 'trophic_lvl_estimate_max', 'trophic_lvl',
    'wb_ph_min', 'wb_ph_max', 'wb_salinity_min', 'wb_salinity_max',
    'wb_do_min', 'wb_do_max', 'wb_bod_min', 'wb_bod_max',
    'wb_turbidity_min', 'wb_turbidity_max', 'wb_temp_min', 'wb_temp_max'
]

In [ ]:
# -----------------------
# 3. Preprocessor
# -----------------------
numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median'))])
low_card_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
high_card_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('target', ce.TargetEncoder())  # category_encoders TargetEncoder
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('low_cat', low_card_transformer, low_cardinality),
    ('high_cat', high_card_transformer, high_cardinality)
], remainder='drop')


In [ ]:
# -----------------------
# 4. Risk discretization
# -----------------------
def discretize_risk(y):
    bins = [0, 0.33, 0.66, 1.0]
    labels = [0, 1, 2]  # 0=Low, 1=Medium, 2=High
    return pd.cut(y, bins=bins, labels=labels, include_lowest=True).astype(int)


In [ ]:
# -----------------------
# 5. Fitness function for OOA/MOOA (unchanged)
# -----------------------
def fitness_function(params, X, y, preprocessor, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    cv_results = {
        "accuracy": [], "precision": [], "recall": [], "f1": [],
        "roc_auc": [], "logloss": [], "train_accuracy": []
    }

    for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        y_tr_class = discretize_risk(y_tr)
        y_val_class = discretize_risk(y_val)

        # Preprocess: fit on train (with labels for target encoder) and transform
        X_tr_num = preprocessor.fit_transform(X_tr, y_tr_class)
        X_val_num = preprocessor.transform(X_val)

        # SMOTE on processed numeric/encoded features
        smote = SMOTE(random_state=SEED)
        X_tr_res, y_tr_res = smote.fit_resample(X_tr_num, y_tr_class)

        # Train XGBoost
        model = xgb.XGBClassifier(
            n_estimators=int(params["n_estimators"]),
            learning_rate=params["learning_rate"],
            max_depth=int(params["max_depth"]),
            subsample=params["subsample"],
            colsample_bytree=params["colsample_bytree"],
            gamma=params["gamma"],
            reg_lambda=params["reg_lambda"],
            objective="multi:softprob",
            eval_metric="mlogloss",
            min_child_weight=int(params["min_child_weight"]),
            reg_alpha=params["reg_alpha"],
            scale_pos_weight=params["scale_pos_weight"],
            use_label_encoder=False,
            random_state=SEED,
            verbosity=0
        )

        model.fit(
            X_tr_res, y_tr_res,
            eval_set=[(X_val_num, y_val_class)],
            verbose=False
        )

        # Predictions
        y_pred_val = model.predict(X_val_num)
        y_pred_tr = model.predict(X_tr_res)
        y_val_probs = model.predict_proba(X_val_num)

        # Metrics
        cv_results["accuracy"].append(accuracy_score(y_val_class, y_pred_val))
        cv_results["precision"].append(precision_score(y_val_class, y_pred_val, average="weighted", zero_division=0))
        cv_results["recall"].append(recall_score(y_val_class, y_pred_val, average="weighted"))
        cv_results["f1"].append(f1_score(y_val_class, y_pred_val, average="weighted"))
        cv_results["roc_auc"].append(roc_auc_score(pd.get_dummies(y_val_class), y_val_probs, multi_class="ovr"))
        cv_results["logloss"].append(log_loss(pd.get_dummies(y_val_class), y_val_probs))
        cv_results["train_accuracy"].append(accuracy_score(y_tr_res, y_pred_tr))

    return np.mean(cv_results["roc_auc"]), cv_results

    # Final fitness: validation accuracy penalized by generalization gap
    avg_val_acc = np.mean(cv_results["accuracy"])
    avg_gap = np.mean(cv_results["generalization_gap"])
    fitness_score = avg_val_acc - avg_gap  # Encourage generalizable models

    return fitness_score, cv_results


# -----------------------
# Helper: parameter bounds & utilities for MOOA
# -----------------------
param_bounds = {
    "n_estimators": (500, 2500),
    "learning_rate": (0.005, 0.15),
    "max_depth": (6, 14),
    "subsample": (0.65, 0.95),
    "colsample_bytree": (0.65, 0.95),
    "gamma": (0, 15.0),
    "reg_lambda": (0.1, 40.0),
    "reg_alpha": (0.0, 15.0),
    "min_child_weight": (1, 30),
    "max_delta_step": (0, 20),
    "scale_pos_weight": (0.5, 15.0)
}
int_params = {"n_estimators", "max_depth", "min_child_weight", "max_delta_step"}

def clamp_param(key, val):
    lo, hi = param_bounds[key]
    if key in int_params:
        val = int(round(val))
    return max(lo, min(hi, val))

def agent_to_vector(agent):
    keys = list(param_bounds.keys())
    return np.array([agent[k] for k in keys], dtype=float), keys

def vector_to_agent(vec, keys):
    agent = {}
    for i, k in enumerate(keys):
        if k in int_params:
            agent[k] = int(round(vec[i]))
        else:
            agent[k] = float(vec[i])
        agent[k] = clamp_param(k, agent[k])
    return agent

# -----------------------
# Helper: Lévy (Mantegna) and Brownian steps
# -----------------------
def levy_step(dimension, beta=1.5, scale=0.1):
    # Mantegna's algorithm
    sigma_u = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) /
               (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1.0 / beta)
    u = np.random.normal(0, sigma_u, size=dimension)
    v = np.random.normal(0, 1.0, size=dimension)
    step = u / (np.abs(v) ** (1.0 / beta))
    return scale * step

def brownian_step(dimension, scale=0.02):
    return np.random.normal(0, scale, size=dimension)

# -----------------------
# Helper: RFDB selection (fitness + distance)
# -----------------------
def rfdb_select(agents_vecs, agents_scores, global_best_vec, w_f=0.75, w_d=0.25, eps=1e-12):
    scores = np.array(agents_scores, dtype=float)
    # normalize fitness
    if np.ptp(scores) < eps:
        norm_f = np.ones_like(scores) / len(scores)
    else:
        norm_f = (scores - scores.min()) / (scores.max() - scores.min())
    # distances
    dists = np.linalg.norm(np.stack(agents_vecs) - global_best_vec, axis=1)
    if np.ptp(dists) < eps:
        norm_d = np.ones_like(dists) / len(dists)
    else:
        norm_d = (dists - dists.min()) / (dists.max() - dists.min())
    combined = w_f * norm_f + w_d * norm_d
    combined = combined + eps
    probs = combined / combined.sum()
    idx = np.random.choice(len(probs), p=probs)
    return idx

In [ ]:
# -----------------------
# 6. MOOA Search (modified)
# -----------------------
def run_MOOA(X, y, preprocessor, max_iter=10, n_agents=20, random_state=SEED,
             prob_explore=0.7, levy_prob=0.7, brownian_prob=0.3):
    np.random.seed(random_state)
    random.seed(random_state)

    # initialize agents
    agents = []
    for _ in range(n_agents):
        ag = {
            "n_estimators": np.random.randint(param_bounds["n_estimators"][0], param_bounds["n_estimators"][1] + 1),
            "learning_rate": np.random.uniform(*param_bounds["learning_rate"]),
            "max_depth": np.random.randint(param_bounds["max_depth"][0], param_bounds["max_depth"][1] + 1),
            "subsample": np.random.uniform(*param_bounds["subsample"]),
            "colsample_bytree": np.random.uniform(*param_bounds["colsample_bytree"]),
            "gamma": np.random.uniform(*param_bounds["gamma"]),
            "min_child_weight": np.random.randint(param_bounds["min_child_weight"][0], param_bounds["min_child_weight"][1] + 1),
            "max_delta_step": np.random.randint(param_bounds["max_delta_step"][0], param_bounds["max_delta_step"][1] + 1),
            "reg_alpha": np.random.uniform(*param_bounds["reg_alpha"]),
            "reg_lambda": np.random.uniform(*param_bounds["reg_lambda"]),
            "scale_pos_weight": np.random.uniform(*param_bounds["scale_pos_weight"])
        }
        agents.append(ag)

    best_agent = None
    best_score = -np.inf
    best_cv_results = None

    keys = list(param_bounds.keys())
    agents_scores = [None] * n_agents
    agents_vecs = [None] * n_agents

    # Evaluate initial population
    for i, ag in enumerate(agents):
        vec, _ = agent_to_vector(ag)
        agents_vecs[i] = vec.copy()
        score, cv = fitness_function(ag, X, y, preprocessor)
        agents_scores[i] = score
        if score > best_score:
            best_score = score
            best_agent = deepcopy(ag)
            best_cv_results = cv
    print(f"Init best ROC-AUC = {best_score:.4f}")

    # Main loop
    for it in range(1, max_iter + 1):
        best_vec, _ = agent_to_vector(best_agent)
        new_agents = []
        for i, ag in enumerate(agents):
            vec, _ = agent_to_vector(ag)
            dim = len(vec)

            if np.random.rand() < prob_explore:
                # Phase 1: Exploration
                if np.random.rand() < levy_prob:
                    step = levy_step(dim, beta=1.5, scale=0.08)
                else:
                    step = brownian_step(dim, scale=0.02)
                scaled_step = np.zeros_like(step)
                for j, k in enumerate(keys):
                    lo, hi = param_bounds[k]
                    rng = hi - lo
                    scaled_step[j] = step[j] * rng
                new_vec = vec + scaled_step * (np.random.rand(dim) * 0.5 + 0.5)
            else:
                # Phase 2: Exploitation (RFDB)
                idx_partner = rfdb_select(agents_vecs, agents_scores, best_vec, w_f=0.75, w_d=0.25)
                partner_vec = agents_vecs[idx_partner].copy()
                w1 = np.random.rand()
                w2 = np.random.rand()
                new_vec = vec + w1 * (partner_vec - vec) + w2 * (best_vec - vec)
                new_vec = new_vec + brownian_step(dim, scale=0.005) * np.array([param_bounds[k][1] - param_bounds[k][0] for k in keys])

            candidate_agent = vector_to_agent(new_vec, keys)
            score_candidate, cv_candidate = fitness_function(candidate_agent, X, y, preprocessor)

            accept = False
            if agents_scores[i] is None or score_candidate > agents_scores[i]:
                accept = True
            else:
                if np.random.rand() < 0.05:
                    accept = True

            if accept:
                new_agents.append(candidate_agent)
                agents_scores[i] = score_candidate
                agents_vecs[i] = agent_to_vector(candidate_agent)[0]
                if score_candidate > best_score:
                    best_score = score_candidate
                    best_agent = deepcopy(candidate_agent)
                    best_cv_results = cv_candidate
            else:
                new_agents.append(ag)

        agents = new_agents
        print(f"Iteration {it}/{max_iter}, Best ROC-AUC = {best_score:.4f}")

    return best_agent, best_score, best_cv_results


In [ ]:
# -----------------------
# 7. Run MOOA
# -----------------------
start_time = time.time()
best_params, best_score, best_cv_results = run_MOOA(
    train_set.drop(columns=["invasion_risk_score", "status"]),
    train_set["invasion_risk_score"],
    preprocessor,
    max_iter=20,
    n_agents=20,
    random_state=SEED
)
end_time = time.time()

print("\nBest MOOA parameters:")
print(best_params)

# Fold-wise results
metrics = ["accuracy", "precision", "recall", "f1", "roc_auc", "logloss"]
for metric in metrics:
    print(f"\n{metric.upper()}:")
    for i, score in enumerate(best_cv_results[metric], 1):
        print(f"Fold-{i}: {score:.4f}")
    print(f"Mean {metric.upper()}: {np.mean(best_cv_results[metric]):.4f}")

# Generalization gap
gen_gap = np.array(best_cv_results["train_accuracy"]) - np.array(best_cv_results["accuracy"])
print("\nGENERALIZATION GAP (Train - Validation Accuracy) per fold:")
for i, gap in enumerate(gen_gap, 1):
    print(f"Fold-{i}: {gap:.4f}")
print(f"Mean Generalization Gap: {np.mean(gen_gap):.4f}")

# Timing stats
total_time = end_time - start_time
num_iterations = 15
time_per_iteration = total_time / num_iterations
val_scores = best_cv_results["accuracy"]
variance = np.var(val_scores)

print("\nEXPERIMENT 2 - TUNING STATISTICS")
print(f"Total tuning time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
print(f"Number of iterations/folds: {num_iterations}")
print(f"Time per iteration: {time_per_iteration:.2f} seconds")
print(f"Validation score mean: {np.mean(val_scores):.4f}")
print(f"Validation score variance: {variance:.6f}")


Init best ROC-AUC = 0.9322
Iteration 1/20, Best ROC-AUC = 0.9345
Iteration 2/20, Best ROC-AUC = 0.9345


In [ ]:
# -----------------------
# 8. Final Train on full train_set + Test eval
# -----------------------
X_train_full = train_set.drop(columns=["invasion_risk_score", "status"])  # drop status too
y_train_full = discretize_risk(train_set["invasion_risk_score"])

# Fit preprocessor on full training data (so target encoder sees full training labels)
X_train_full_num = preprocessor.fit_transform(X_train_full, y_train_full)

# Apply SMOTE
smote = SMOTE(random_state=SEED)
X_train_res, y_train_res = smote.fit_resample(X_train_full_num, y_train_full)

# Final model with best MOOA params
final_model = xgb.XGBClassifier(
    n_estimators=int(best_params["n_estimators"]),
    learning_rate=best_params["learning_rate"],
    max_depth=int(best_params["max_depth"]),
    subsample=best_params["subsample"],
    colsample_bytree=best_params["colsample_bytree"],
    gamma=best_params["gamma"],
    min_child_weight=int(best_params["min_child_weight"]),
    max_delta_step=int(best_params["max_delta_step"]),
    reg_alpha=best_params["reg_alpha"],
    reg_lambda=best_params["reg_lambda"],
    scale_pos_weight=best_params["scale_pos_weight"],
    objective="multi:softprob",
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state=SEED,
    verbosity=0
)
final_model.fit(X_train_res, y_train_res)


KeyError: 'reg_alpha'

In [ ]:
# -----------------------
# Train set evaluation (on resampled training set)
# -----------------------
y_train_pred = final_model.predict(X_train_res)
y_train_probs = final_model.predict_proba(X_train_res)

print("\nFinal Training Performance (with SMOTE):")
print("Accuracy:", accuracy_score(y_train_res, y_train_pred))
print("Precision:", precision_score(y_train_res, y_train_pred, average="weighted", zero_division=0))
print("Recall:", recall_score(y_train_res, y_train_pred, average="weighted"))
print("F1:", f1_score(y_train_res, y_train_pred, average="weighted"))
print("ROC-AUC:", roc_auc_score(pd.get_dummies(y_train_res), y_train_probs, multi_class="ovr"))
print("Logloss:", log_loss(pd.get_dummies(y_train_res), y_train_probs))



Final Training Performance (with SMOTE):
Accuracy: 0.9937555753791257
Precision: 0.9937593145793698
Recall: 0.9937555753791257
F1: 0.9937555645717745
ROC-AUC: 0.9995797991099553
Logloss: 0.4678019989316506


In [ ]:
# -----------------------
# Test set evaluation
# -----------------------
X_test = test_set.drop(columns=["invasion_risk_score", "status"])
y_test = discretize_risk(test_set["invasion_risk_score"])

# Use preprocessor already fit on full training data
X_test_num = preprocessor.transform(X_test)

y_test_pred = final_model.predict(X_test_num)
y_test_probs = final_model.predict_proba(X_test_num)

print("\nTest Performance:")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred, average="weighted", zero_division=0))
print("Recall:", recall_score(y_test, y_test_pred, average="weighted"))
print("F1:", f1_score(y_test, y_test_pred, average="weighted"))
try:
    print("ROC-AUC:", roc_auc_score(pd.get_dummies(y_test), y_test_probs, multi_class="ovr"))
except Exception as e:
    print("ROC-AUC calculation failed:", e)
print("Logloss:", log_loss(pd.get_dummies(y_test), y_test_probs))

print("\nClassification report (test):")
print(classification_report(y_test, y_test_pred, zero_division=0))
print("Confusion matrix (test):")
print(confusion_matrix(y_test, y_test_pred))


Test Performance:
Accuracy: 0.7589605734767025
Precision: 0.8965474961575878
Recall: 0.7589605734767025
F1: 0.8045850365107602
ROC-AUC: 0.9329776490521436
Logloss: 0.7297640752276385

Classification report (test):
              precision    recall  f1-score   support

           0       0.94      0.73      0.82       416
           1       0.23      0.87      0.36        76
           2       0.95      0.76      0.85       624

    accuracy                           0.76      1116
   macro avg       0.71      0.79      0.68      1116
weighted avg       0.90      0.76      0.80      1116

Confusion matrix (test):
[[305  94  17]
 [  3  66   7]
 [ 18 130 476]]
